<a href="https://colab.research.google.com/github/itz-A53-K/rookie_dev/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_groq langchain_core langchain-community
!pip install chromadb
!pip install flask flask_cors pyngrok
!pip install pypdf
!pip install django
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 32.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux20

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os

from transformers import pipeline



# Initialize sentiment analysis model
sentiment_analyzer = pipeline("sentiment-analysis")

# Initialize the LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # Replace with your actual key
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# Create or load the vector database
def create_vector_db():
    loader = DirectoryLoader("/content/drive/MyDrive/Data", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# Setup QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """
     Make the answers as humane as possible and make the answers as short as possible
     and don't answer questions other than Mental Health Questions or Physical health Questions.
     You can answer which doctor to consult based on provided symptoms and
     if its a mental health try to answer them humanely
     and write only you don't have to recommend any doctor in that case.":
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Initialize the chatbot
print("Initializing Chatbot...")
llm = initialize_llm()

db_path = "./chroma_db"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

# Run chatbot in CLI
def chat_cli():
    print("Chatbot is ready! Type 'exit' to quit.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Sentiment analysis
        sentiment = sentiment_analyzer(user_input)[0]
        sentiment_label = sentiment['label']
        sentiment_score = sentiment['score']

        # Generate response
        result = qa_chain.run(user_input)

        print(f"Chatbot ({sentiment_label} - {sentiment_score:.2f}): {result}")

if __name__ == '__main__':
    chat_cli()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Initializing Chatbot...


<ipython-input-3-38bfb34f9055>:32: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

<ipython-input-3-38bfb34f9055>:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


ChromaDB created and data saved
Chatbot is ready! Type 'exit' to quit.

You: hi


<ipython-input-3-38bfb34f9055>:88: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(user_input)


Chatbot (POSITIVE - 1.00): How are you feeling today?

You: bad
Chatbot (NEGATIVE - 1.00): You're feeling bad. That can be really tough. Would you like to talk about what's on your mind?

You: I want to consult a doctor
Chatbot (NEGATIVE - 0.94): What seems to be bothering you, physically or emotionally?

You: emotionally
Chatbot (POSITIVE - 0.99): You're feeling overwhelmed and struggling with difficult emotions. That's okay, it's normal to feel that way sometimes.


KeyboardInterrupt: Interrupted by user

In [ ]:
ngrok

<module 'pyngrok.ngrok' from '/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py'>

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
import json
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Initialize Flask App
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Initialize the LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # Replace with your actual key
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# Create or load the vector database
def create_vector_db():
    loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# Setup QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """
     Make the answers as humane as possible and make the answers as short as possible
     and don't answer questions other than Mental Health Questions or Physical health Questions.
     You can answer which doctor to consult based on provided symptoms and
     if its a mental health try to answer them humanely
     and write only you don't have to recommend any doctor in that case.:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Initialize the chatbot
print("Initializing Chatbot...")
llm = initialize_llm()

db_path = "./chroma_db"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

# Prediction function using LangChain model
@app.route('/predict', methods=['POST'])
def predict_route():
    data = request.get_json()
    user_input = data.get("message", "")
    result = qa_chain.run(user_input)  # Calls your LangChain chatbot
    return jsonify({"response": result})

# Expose API via ngrok
NGROK_AUTH_TOKEN = "2tcplzFDBKMQn6gnr4TL7GB5wnr_4BciG7g447hzCkYnU4Tkm"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)

# External Google Colab API URL (replace with your ngrok URL)
COLAB_API_URL = public_url + "/predict"

# Django API endpoint for chatbot
@csrf_exempt
def chat_with_bot(request):
    if request.method == 'POST':
        try:
            data = json.loads(request.body)
            user_input = data.get('message', '')

            print("Sending request to:", COLAB_API_URL)  # Debugging log
            print("User Input:", user_input)

            response = requests.post(COLAB_API_URL, json={"message": user_input})

            print("Colab API Status:", response.status_code)
            print("Colab API Response:", response.text)

            if response.status_code == 200:
                return JsonResponse(response.json())
            else:
                return JsonResponse({"error": "Failed to connect to Colab API"}, status=500)

        except Exception as e:
            print("Error in Django API:", str(e))  # Debugging log
            return JsonResponse({"error": "Internal Server Error"}, status=500)

    return JsonResponse({"error": "Invalid request"}, status=400)

Initializing Chatbot...
Public URL: https://ccbe-34-121-131-93.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
import json
import requests
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Initialize Flask App
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Initialize the LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # Replace with your actual key
        model_name="llama-3.3-70b-versatile"
    )
    return llm

# Create or load the vector database
def create_vector_db():
    loader = DirectoryLoader("./data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

# Setup QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """
     Make the answers as humane as possible and make the answers as short as possible
     and don't answer questions other than Mental Health Questions or Physical health Questions.
     You can answer which doctor to consult based on provided symptoms and
     if its a mental health try to answer them humanely
     and write only you don't have to recommend any doctor in that case.:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=['context', 'question'])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

# Initialize the chatbot
print("Initializing Chatbot...")
llm = initialize_llm()

db_path = "./chroma_db"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

# Prediction function using LangChain model
@app.route('/predict', methods=['POST'])
def predict_route():
    data = request.get_json()
    user_input = data.get("message", "")
    result = qa_chain.run(user_input)  # Calls your LangChain chatbot
    return jsonify({"response": result})

# Expose API via ngrok
NGROK_AUTH_TOKEN = "2tcplzFDBKMQn6gnr4TL7GB5wnr_4BciG7g447hzCkYnU4Tkm"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)

# External Google Colab API URL (replace with your ngrok URL)
COLAB_API_URL = public_url + "/predict"

# Django API endpoint for chatbot
user_sessions = {}  # Store conversation history

@csrf_exempt
def chat_with_bot(request):
    if request.method == 'POST':
        try:
            data = json.loads(request.body)
            user_id = data.get('user_id', 'default')  # Unique identifier
            user_input = data.get('message', '')

            # Retrieve previous context
            context = user_sessions.get(user_id, "")

            # Append new message to context
            user_sessions[user_id] = context + f"User: {user_input}\n"

            # Send query with context
            response = requests.post(COLAB_API_URL, json={"message": user_sessions[user_id]})

            if response.status_code == 200:
                return JsonResponse(response.json())
            else:
                return JsonResponse({"error": "Failed to connect to Colab API"}, status=500)

        except Exception as e:
            return JsonResponse({"error": "Internal Server Error"}, status=500)

    return JsonResponse({"error": "Invalid request"}, status=400)


Initializing Chatbot...
Public URL: https://ce38-34-105-54-130.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 03:50:29] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 03:52:08] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 03:59:39] "POST /predict/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:00:31] "POST /predict/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:00:51] "POST /predict/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:01:17] "POST /predict/ HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:01:37] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:02:08] "POST /predict HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:02:57] "POST /predict HTTP/1.1" 415 -
INFO:werkzeug:127.0.0.1 - - [28/Feb/2025 04:04:03] "POST /pred